In [1]:
#import packages

import numpy as np
import pandas as pd
import random
import pickle
import itertools
import folium #mapping
import seaborn as sns #color palette
#from colormap import rgb2hex #chnage RGB codes to hex codes

In [2]:
## USE THIS CODE TO IMPORT DATA

file_name = '../data/results/results_cement.pkl' ##results_toydata_uniform.pkl'
open_file = open(file_name, "rb")
loaded_list = pickle.load(open_file)
open_file.close()

In [3]:
#convert to dataframe

df = pd.DataFrame(loaded_list, columns = ['supplier', 'receiver', 'distance', 'volume'])
df.head()

#convert suppliers to list to get rid of weird numpy object datatype
supplier_split = df.supplier.to_list()
receiver_split = df.receiver.to_list()
#supplier_split[0:5]


#convert suppliers to lat and lon
supplier_lat= [supplier_split[j][0] for j in range(0,len(supplier_split))]
supplier_lon = [supplier_split[j][1] for j in range(0,len(supplier_split))]

#convert to list of receiver lat and long
receiver_lat= [receiver_split[j][0] for j in range(0,len(receiver_split))]
receiver_lon = [receiver_split[j][1] for j in range(0,len(receiver_split))]

#add back into the df
df['supplier_lat'] = supplier_lat
df['supplier_lon'] = supplier_lon
df['receiver_lat'] = receiver_lat
df['receiver_lon'] = receiver_lat

df.head()

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon
0,"[35.029298, -118.316236]","[36.24231905, -120.3481971]",266375.614415,1613.9108,35.029298,-118.316236,36.242319,36.242319
1,"[35.029298, -118.316236]","[36.24957886, -120.3470871]",152376.412122,1727.4278,35.029298,-118.316236,36.249579,36.249579
2,"[35.029298, -118.316236]","[36.24632199, -120.35282340000002]",500960.004462,1534.2520,35.029298,-118.316236,36.246322,36.246322
3,"[35.029298, -118.316236]","[36.24188108, -120.3498502]",458196.753679,1462.9921,35.029298,-118.316236,36.241881,36.241881
4,"[35.029298, -118.316236]","[36.24289301, -120.345552]",442842.047134,1448.0971,35.029298,-118.316236,36.242893,36.242893


In [4]:
#create a color palette, repeat it 100 times to see how many things I can visualize at once in the html
palette = ['red', 'green', 'purple', 'orange',  'darkblue',
           'gray','cadetblue', 'darkpurple', 'white', 'pink', 'lightblue', 'lightgreen',
           'black','lightgray','lightred','blue','beige']*100
            #‘lightgreen’] ‘gray’, ‘black’, ‘lightgray’] 'darkred','lightred','blue','beige','white',


#palette

## Map Everything

Shape of dataframe

In [5]:
df.shape

(29376, 8)

In [6]:
#view a record
df[df['supplier_lat'] == 35.399380]

,supplier,receiver,distance,volume,supplier_lat,supplier_lon,receiver_lat,receiver_lon


Get counts by supplier

In [7]:
#counts by each supplier
df_counts = df.groupby(['supplier_lat','supplier_lon']).count().reset_index()
df_sorted = df_counts.sort_values('distance')
df_sorted.tail(20)

,supplier_lat,supplier_lon,supplier,receiver,distance,volume,receiver_lat,receiver_lon
1,34.604500,-117.338200,582,582,582,582,582,582
4,35.029298,-118.316236,691,691,691,691,691,691
3,34.819863,-118.748732,1985,1985,1985,1985,1985,1985
6,40.736900,-122.322300,2462,2462,2462,2462,2462,2462
0,34.437557,-116.891034,2682,2682,2682,2682,2682,2682
5,37.318100,-122.091000,5402,5402,5402,5402,5402,5402
2,34.622200,-117.100100,15572,15572,15572,15572,15572,15572


Show the 10 suppliers with the shortest mean distance

In [8]:
#group the dataframe by mean distance
df_grouped = df.groupby(['supplier_lat','supplier_lon']).mean('distance').reset_index()
df_grouped

,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
0,34.437557,-116.891034,1.964894e+06,706.589504,35.449529,35.449529
1,34.604500,-117.338200,4.200970e+06,2149.011409,34.824101,34.824101
2,34.622200,-117.100100,3.930772e+06,432.301662,35.517251,35.517251
3,34.819863,-118.748732,3.008837e+06,776.561685,35.452441,35.452441
4,35.029298,-118.316236,1.010611e+06,1625.738553,35.478622,35.478622
5,37.318100,-122.091000,4.294457e+06,701.619048,35.510165,35.510165
6,40.736900,-122.322300,6.886070e+06,769.729101,35.464503,35.464503


In [9]:
#sort on distance
df_grouped = df_grouped.sort_values('distance')
df_grouped

,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
4,35.029298,-118.316236,1.010611e+06,1625.738553,35.478622,35.478622
0,34.437557,-116.891034,1.964894e+06,706.589504,35.449529,35.449529
3,34.819863,-118.748732,3.008837e+06,776.561685,35.452441,35.452441
2,34.622200,-117.100100,3.930772e+06,432.301662,35.517251,35.517251
1,34.604500,-117.338200,4.200970e+06,2149.011409,34.824101,34.824101
5,37.318100,-122.091000,4.294457e+06,701.619048,35.510165,35.510165
6,40.736900,-122.322300,6.886070e+06,769.729101,35.464503,35.464503


In [10]:
#cut off at the top 10
df_grouped = df_grouped.reset_index() #10 at the .tail(10) head(10)
df_grouped

,index,supplier_lat,supplier_lon,distance,volume,receiver_lat,receiver_lon
0,4,35.029298,-118.316236,1.010611e+06,1625.738553,35.478622,35.478622
1,0,34.437557,-116.891034,1.964894e+06,706.589504,35.449529,35.449529
2,3,34.819863,-118.748732,3.008837e+06,776.561685,35.452441,35.452441
3,2,34.622200,-117.100100,3.930772e+06,432.301662,35.517251,35.517251
4,1,34.604500,-117.338200,4.200970e+06,2149.011409,34.824101,34.824101
5,5,37.318100,-122.091000,4.294457e+06,701.619048,35.510165,35.510165
6,6,40.736900,-122.322300,6.886070e+06,769.729101,35.464503,35.464503


In [11]:
df_small = pd.DataFrame()

#loop through and get the full dataset. can't group by the lat/long because it's some unhashable type.
for i in range(0,df_grouped.shape[0]):
    this_lat = df_grouped['supplier_lat'][i]
    this_lon = df_grouped['supplier_lon'][i]
    receivers = df[(df['supplier_lat'] == this_lat) & (df['supplier_lon'] == this_lon)]
    df_small = df_small.append(receivers)
    
df_small.shape

(29376, 8)

In [12]:
#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers.shape
#unique_suppliers['supplier_lon'][0]

(7, 2)

In [19]:
vols = df['volume']
sum(vols)

18227678.59586229

In [14]:
#determine the maximum volume - this will be used to scale the line weight
volumes = df['volume']
median_volume = np.median(volumes)
median_volume

volumes = df['volume']
quantiles = np.quantile(volumes, [0.2,0.4,0.6,0.8])

In [15]:
quantiles

array([ 136.54856,  365.09186,  666.72613, 1051.7717 ])

In [16]:
#create map

#center the map in Cali
#mapit = folium.Map( location=[39.4598465 , -121.94542694], zoom_start=20 )
receiver_count = 0
 
#plot all the suppliers 
latlon = df_small['supplier']
mapit = folium.Map( location=[37.3427, -119.2244], zoom_start=6)

#find unique suppliers for looping and filtering
unique_suppliers = df_small[['supplier_lat', 'supplier_lon']]
unique_suppliers = unique_suppliers.drop_duplicates()
unique_suppliers =unique_suppliers.reset_index(drop=True)
unique_suppliers

#loop through suppliers, paint all the receivers in the same color
for i in range(0,unique_suppliers.shape[0]):
    
    #big circle for supplier
    folium.CircleMarker( location=[ unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i] ], fill=True, 
                        fill_color=palette[i], color=palette[i],  radius=5 ).add_to( mapit ) #'
    
    #find all receivers associated with the supplier late and lon, filter the df
    this_lat = unique_suppliers['supplier_lat'][i]
    this_lon = unique_suppliers['supplier_lon'][i]
    receivers = df_small[(df_small['supplier_lat'] == this_lat) & (df_small['supplier_lon'] == this_lon)]
    
    #plot the receivers with a new color each time
    latlon = receivers['receiver']
    #print(receivers.shape[0])
    #print(receivers.shape[1])
    receiver_count += receivers.shape[0] #verify 1000
    
    #for each receiver  
    for j in range(0,len(latlon)): #coord in latlon: 
        
        lat = latlon.iloc[j][0]
        lon = latlon.iloc[j][1]
        
        volume = receivers.iloc[j]['volume']
        if volume > quantiles[3]:
            quantile = 1 #80-100 percentile
        elif volume > quantiles[2]:
            quantile = 0.8 #60-80 percentile
        elif volume > quantiles[1]:
            quantile = 0.6 #40-60 percentile
        elif volume > quantiles[0]:
            quantile = 0.4 #20-40 percentile
        else:
            quantile  = 0.2 #0-20 percentile
        line_weight = quantile * 3
        #print(line_weight)
        #print(volume, quantile)
        
        #line coordinates
        line_coords = [ [unique_suppliers['supplier_lat'][i],unique_suppliers['supplier_lon'][i]],
                      [lat, lon] ]#[coord[0], coord[1] ]  ]
        my_PolyLine= folium.PolyLine(locations=line_coords,color=palette[i],weight=line_weight) #color and supplier use i index
        mapit.add_child(my_PolyLine)
        
        #dot for the receiver
        folium.CircleMarker( location=[ lat,lon ], fill=True, fill_color=palette[i], color=palette[i],  radius=1 ).add_to( mapit )

print(receiver_count)
mapit

mapit.save(outfile= "visualization.html")

29376
